# SQL with Python

#### OOPs concepts are extensively used to increase reusablity of code
#### Every Step is logged into a file using custom logging class

## Note: Credits of this work goes to Arjun Panwar, you may check his pages here:
## LinkedIn: https://www.linkedin.com/in/arjun-panwar/
## GitHub: https://github.com/arjun-panwar

## Data Description

Data Set used: Carbon Nanotubes

Data Set URL: https://archive.ics.uci.edu/ml/datasets/Carbon+Nanotubes

Number of Instances: 10721

#### I will provide the dataset in the repository

## Logging Class

In [1]:
from datetime import datetime #importing DateTime package
class App_Logger:
    '''
    It is used save logs into a file
    Parameters
    ----------
    file: log file name Default is logfile.log
    '''
    
    def __init__(self,file="logfile.log"):
        self.f_name = file
    
    def log(self,log_type,log_msg):
        '''
        Function log to save logs and log type in file
        Parameters
        ----------
        log_type: Type of log-info,error,warning etc
        log_msg: Log to be saved(message)
        '''
        now = datetime.now() #current time
        current_time = now.strftime("%d-%m-%Y %H:%M:%S") #changing time formation
        f= open(self.f_name, "a+") #opening file in append + mode
        f.write(current_time+","+log_type+","+log_msg+"\n") #writing log type
        f.close() #closing log file

## SQL Class

In [18]:
!pip install mysql-connector-python-rf

  Created wheel for mysql-connector-python-rf: filename=mysql_connector_python_rf-2.2.2-cp39-cp39-win_amd64.whl size=249453 sha256=820afeab5b2dd0b126c8d8ba3102f3f25172f576258da898e1b54cd1b92a4cf7
  Stored in directory: c:\users\y\appdata\local\pip\cache\wheels\64\06\5b\ef9543936a3f5de15b02775b6ae548a09f3ed2319d20342771
Successfully built mysql-connector-python-rf


In [2]:
import mysql.connector as connection #importing MYSQL connector
import pandas as pd #importing pandas
class sql:
    '''
    SQL class through with we can perform most of the SQL tasks using python
    Parameters
    ----------
    host: host URL of MySQL server
    user: user name
    passwd: password
    db: database name- default empty string ("")
    '''
    
    def __init__(self,host,user,passwd,db=""):
        '''
        init function of sql class
        '''
        self.host=host
        self.user=user
        self.passwd=passwd
        self.db=db
        self.logger=App_Logger("logfile.txt") #creating App_Logger object
        self.logger.log("info", "SQL object created") #logging
        
    def conn(self):
        '''
        Function conn is used to make connection to SQL server
        Parameters
        ----------
        '''
        try:
            if self.db=="":
                #connection without db
                return connection.connect(host=self.host,user=self.user,passwd=self.passwd)
            else:
                # connection with db
                return connection.connect(host=self.host,user=self.user, data=self.db)
        except Exception as e:
            self.logger.log("error", f"connection error : {str(e)}") #logging
            print(str(e))
                                          
    def db_list(self):
        '''
        Function db_list is used to show databases list
        Parameters
        ----------
        '''
        try:
            conn=self.conn() #making connection
            cursor = conn.cursor() #create a cursor to execute queries
            q="SHOW DATABASES" #qyery
            cursor.execute(q) #executing Query
            print(cursor.fetchall()) #printing result
            conn.close() #connection closed
            self.logger.log("info", "DB list displayed") #logged
                                          
        except Exception as e:
            conn.close()#connection closed
            print(str(e))
            self.logger.log("error", f"db list error : {str(e)}") #logging
                                          
    def create_db(self,db_name):
        '''
        Function create_ db is used to create a new database
        Parameters
        ----------
        db_name: database name
        '''
        try:
            conn=self.conn() #making connection
            cursor = conn.cursor() #create a cursor to execute queries
            cursor.execute(f"create database {db_name}") #executing Query
            self.db=db_name #Initializing database name to class variable so
            conn.close()#connection closed
            self.logger.log("info", f"{db_name} DB created") #logging
                                          
        except Exception as e:
            conn.close()#connection closed
            print(str(e))
            self.logger.log("error", f"db not created error : {str(e)}") #log
                                          
    def drop_db(self,db_name):
        '''
        Function drop_ db is used to drop a database
        Parameters
        ----------
        db_name: database name
        '''
        try:
            conn=self.conn() #making connection
            cursor = conn.cursor() #create a cursor to execute queries
            cursor.execute(f"drop database {db_name}") #executing Query
            conn.close() #connection closed
            self.logger.log("info", f"{db_name} DB droped") #logging
                                          
        except Exception as e:
            conn.close()#connection closed
            print(str(e))
            self.logger.log("error", f"db not Droped error : {str(e)}") #logg
                                          
    def create_table(self,table_name,columns):
        '''
        Function create_ table is used to create a new table
        Parameters
        ----------
        table_name: table name
        columns: columns names with data type and other discription in SQL fo
        '''
        try:
            conn=self.conn() #making connection
            cursor = conn.cursor() #create a cursor to execute queries
            cursor.execute(f"CREATE TABLE {table_name} ({columns})") #executi
            conn.close() #connection closed
            self.logger.log("info", f"{table_name} table created with columns")
                            
        except Exception as e:
            conn.close() #connection closed
            print(str(e))
            self.logger.log("error", f"table not created error : {str(e)}") #
                
    def insert(self,table_name,data):
        '''
        Function insert is used to insert value in table
        Parameters
        ----------
        table_name: table name
        data: values to be inserted
        '''
        try:
            conn=self.conn() #making connection
            cursor = conn.cursor() #create a cursor to execute queries
            cursor.execute(f"INSERT INTO {table_name} VALUES ({data})") #exec
            conn.commit() #commiting the query
            conn.close() #connection closed
                            
        except Exception as e:
            conn.close() #connection closed
            self.logger.log("error", f"insert error : {str(e)}") #logging
                            
    def dump_file(self,f_name,t_name,columns,csv=True):
        '''
        Function dump_file is used to dump a csv into a table
        Parameters
        ----------
        f_name: file name
        t_name: table name
        columns: columns names with data type and other discription in SQL f
        csv: True if csv file is comma separated otherwise False if csv file
        '''
        try:
            f=open(f_name,"r") #opening file in read mode
            f.readline() #reading first line to skip columns line in file
            self.create_table(t_name,columns) #creating table
            for line in f.readlines(): #reading file line by line
                if csv:
                    data="\'"+line[:-1].replace(",","\',\'")+"\'" # data for
                    print(data)
                else:
                    data="\'"+line[:-1].replace(";","\',\'")+"\'" # data form
                self.insert(t_name,data) #inserting data
            self.logger.log("info", f"{f_name} file data dumped to {t_name} t")
                            
        except Exception as e:
            print(str(e))
            self.logger.log("error", f"file dump error : {str(e)}") #logging
                            
    def select_db(self,db_name):
        '''
        Function select_db is used to select a database
        Parameters
        ----------
        db_name: database name
        '''
        self.db=db_name #Initializing database name to class variable so that
        self.logger.log("info", f"{db_name} DB selected") #logging
                            
    def columns(self,t_name):
        '''
        Function columns is used to print columns names
        Parameters
        ----------
        t_name: table name
        '''
        try:
            conn=self.conn() #making connection
            cursor = conn.cursor() #create a cursor to execute queries
            cursor.execute(f"SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE T")
            for result in cursor.fetchall(): #printing result
                print(result[3],end=",")
            conn.close() #connection closed
            self.logger.log("info", f"columns names displayed") #logging
                           
        except Exception as e:
            conn.close() #connection closed
            self.logger.log("error", f"columns name not displayed : {str(e)}")

    def query(self,query):
        '''
        Function query is used to run a SQL query
        Parameters
        ----------
        query: SQL query
        '''
        try:
            conn=self.conn() #making connection
            cursor = conn.cursor() #create a cursor to execute queries
            cursor.execute(query) #executing Query
            for result in cursor.fetchall(): #printing result line by line
                print(result)
            conn.close() #connection closed
            self.logger.log("info", f"Query is performed : {query} ") #loggin
                            
        except Exception as e:
            conn.close() #connection closed
            self.logger.log("error", f"Query not performed : {query} : {str(e)}")

    def pd_query(self,query,h=5,t=5):
        '''
        Function pd_query is used to run a SQL query using pandas
        I used it here to get better output format of table
        Parameters
        ----------
        query: SQL query
        h: no. of results from head - default=5
        t: no. of results from tail - default=5
        '''
        try:
            df=pd.read_sql_query(query,self.conn()) #executing Query using pa
            print("Head") #printing head of dataframe
            print(df.head(h))
            if t>0: #printing Tail of dataframe if tail count is more t
                print("Tail")
                print(df.tail(t))
            self.logger.log("info", f"Query is performed : {query} ") #loggin

        except Exception as e:
            print(str(e))
            self.logger.log("error", f"Query not performed : {query} : {str(e)}")

## Creating Class Object by passing HOST,USER,PASSWORD

In [3]:
ob=sql("localhost","root","2001")

## Selecting DB UCI

In [9]:
ob.select_db("UCI")

## Dumping CSV File

In [ ]:
columns="CI_n INT(2),CI_m INT(2),IAC_u VARCHAR(10),IAC_v VARCHAR(10),IAC_w VARCHAR(10), CAC_u VARCHAR(10), CAC_v VARCHAR(10), CAC_w VARCHAR(10)"
ob.dump_file("carbon_nanotubes.csv","test5",columns)

In [ ]:
# columns names
ob.columns("test5")

## The SQL SELECT QUERY

In [ ]:
#Select all columns
ob.pd_query("Select * from test5")

In [ ]:
#Select IAC_u column
ob.pd_query("Select IAC_u from test5")

In [ ]:
#The SQL SELECT DISTINCT
ob.pd_query("Select Distinct CI_m from test5")

In [ ]:
#The SQL WHERE CLAUSE
ob.pd_query("Select CI_n,IAC_u from test5 where CI_n=2")

In [ ]:
#A WHERE clause with AND
ob.pd_query("Select CI_n,CI_m,IAC_u from test5 where CI_n=2 and CI_m=1")

In [ ]:
#A WHERE clause with OR:
ob.pd_query("Select CI_n,IAC_u from test5 where CI_n=2 or CI_n=3")

In [ ]:
#A WHERE clause with NOT
ob.pd_query("Select CI_n,CI_m,IAC_u from test5 where NOT CI_n=4 and CI_m=3")

In [ ]:
#Order By in Descending order
ob.pd_query("Select CI_n,CI_m,IAC_u from test5 ORDER BY IAC_u DESC")

In [ ]:
#Order By in Ascending order
ob.pd_query("Select CI_n,CI_m,IAC_u from test5 ORDER BY IAC_u ASC")

## The SQL MIN() AND MAX() FUNCTION

In [ ]:
ob.query("Select MIN(IAC_u) from test5")

In [ ]:
ob.query("Select MAX(IAC_u) from test5")

## The SQL COUNT(), AVG() AND SUM() FUNCTION

In [ ]:
ob.query("Select count(IAC_u) from test5")

In [ ]:
ob.query("Select AVG(CI_m) from test5")

In [ ]:
ob.query("Select sum(CI_m) from test5")

## The SQL LIKE-OPERATOR

In [ ]:
# IAC_u starting with 0,05
ob.pd_query("Select * from test5 WHERE IAC_u LIKE '0,05%'")

In [ ]:
# IAC_u ending with 1105
ob.pd_query("Select * from test5 WHERE IAC_u LIKE '%1105'")

In [ ]:
# IAC_u that have "345" in any position.
ob.pd_query("Select * from test5 WHERE IAC_u LIKE '%345%'")

In [ ]:
# IAC_u that starts with "0,44" and ends with "5":
ob.pd_query("Select * from test5 WHERE IAC_u LIKE '0,44%5'")

## The SQL IN AND NOT IN OPERATORS

In [ ]:
ob.pd_query("Select * from test5 WHERE CI_n IN (5,1)")

In [ ]:
ob.pd_query("Select * from test5 WHERE CI_n between 4 and 5")

## The SQL ALIAS

In [ ]:
ob.pd_query("Select CI_n as CI,IAC_u as IAC from test5")

In [ ]:
ob.pd_query("Select IAC_u from test5 Group By IAC_u ")

In [ ]:
ob.pd_query("Select count(IAC_v) from test5 Group By IAC_v ")

In [ ]:
ob.pd_query("Select IAC_v from test5 Group By IAC_v having count(IAC_v)>3 ")

## The SQL CREATE DATABASE

In [ ]:
ob.create_db("Test_DB")

## DROP DATABASE

In [ ]:
ob.drop_db("Test_DB")

## check NOT NuLL

In [ ]:
ob.select_db("UCI")
ob.pd_query("SELECT * FROM test5 WHERE IAC_u IS NOT NULL")

## ALTER TABLE -ADD and DROP COLUMN

In [ ]:
ob.pd_query("ALTER TABLE test5 ADD email varchar(25)")
ob.pd_query("Select * from test5",5,0)

In [ ]:
ob.pd_query("ALTER TABLE test5 DROP email")
ob.pd_query("Select * from test5",5,0)

## ALTER modify COLUMN

In [ ]:
ob.pd_query("ALTER TABLE test5 MODIFY CI_n int NOT NULL")
ob.pd_query("desc test5",5,0)

## SQL CHECK on CREATE TABLE

In [ ]:
ob.pd_query("CREATE TABLE Persons (ID int NOT NULL, LastName varchar(255) NOT
ob.pd_query("desc Persons",5,0)

In [ ]:
#drop check
ob.pd_query("ALTER TABLE Persons DROP CHECK Age")
ob.pd_query("desc Persons",5,0)

## SQL DEFAULT on CREATE TABLE

In [ ]:
ob.pd_query("CREATE TABLE Person1 (ID int NOT NULL, LastName varchar(255) NOT
ob.pd_query("desc Person1",5,0)

## DROP A DEFAULT CONSTRAINT

In [ ]:
ob.pd_query("ALTER TABLE Person1 ALTER City DROP DEFAULT")
ob.pd_query("desc Person1",5,0)

## CREATE INDEX

In [ ]:
ob.pd_query("CREATE INDEX idex_lastname on Person1 (LastName)")
ob.pd_query("desc Person1",5,0)

## CREATE UNIQUE INDEX

In [ ]:
ob.pd_query("Create UNIQUE INDEX id on Person1 (ID)")
ob.pd_query("desc Person1",5,0)

In [ ]:
#drop Index
ob.pd_query("ALTER TABLE Person1 DROP INDEX idex_lastname")
ob.pd_query("desc Person1",5,0)

## CREATE VIEW Syntax

In [ ]:
ob.pd_query("CREATE VIEW test_view AS SELECT CI_n,IAC_u,IAC_v FROM test5")
ob.pd_query("Select * from test_view")

## DROPPING VIEWS

In [ ]:
ob.pd_query("DROP VIEW test_view")
ob.pd_query("Select * from test_view")

## IF Statement in MySQL

In [ ]:
ob.pd_query("Select CI_n, IF(CI_n>4,'MORE','LESS') from test5")